In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-09-30 18:24:50.757964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-30 18:24:50.757995: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
ds_name = 'mnli'
num_labels = 3
valid_split = 'validation_matched'
test_split = 'validation_mismatched'
runs = 7

db_path = "./mnli_9-29.db"
seed = 0
pool_size = 2000
search_size = 200
warmup_runs = 100
annealing_runs = 2000
wandb_project = 'mnli_search'
wandb_entity = 'johntzwei'

model_card = "bert-base-cased"
pretraining = True
max_steps = 3000
eval_steps = 500
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
training_args = select.SubsetTrainingArguments(num_labels=num_labels,
                                               model_card=model_card,
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(db_path=db_path,
                                                seed=seed, 
                                                data_pool_size=pool_size,
                                                optimal_subset_size=search_size,
                                                warmup_runs=warmup_runs,
                                                annealing_runs=annealing_runs,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [ ]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)

Reusing dataset glue (/home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/392702 [00:00<?, ?ex/s]

In [ ]:
processed_ds

In [ ]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds[valid_split], test_ds=processed_ds[test_split]
)

In [ ]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(db_path)
df = df.sort_values('objective')
idx = json.loads(df.iloc[-1]['indexes'])

In [ ]:
print(idx)
df

In [ ]:
select_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    select_quality.append(quality)

In [ ]:
np.mean(select_quality), np.std(select_quality)

In [ ]:
random_quality = []
for i in range(0, runs):
    wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
    idx = np.random.choice(np.arange(pool_size), size=pool_size)
    train_dataset = processed_ds["train"].shuffle(seed=searching_args.seed).select(idx)
    quality = subset_trainer.train_one_step(train_dataset)
    random_quality.append(quality)
random_quality

In [ ]:
np.mean(random_quality), np.std(random_quality)